6610402230 ศิริสุข ทานธรรม

In [1]:
import numpy as np
import pandas as pd
# from sklearn import neighbors
# from sklearn.model_selection import train_test_split

seed_value = 12345
np.random.seed(seed_value)

def train_test_split(X, Y, test_size):
    X = np.array(X)
    Y = np.array(Y)
    
    index_list = np.arange(X.shape[0])
    np.random.shuffle(index_list)
    
    X = X[index_list]
    Y = Y[index_list]
    
    train_size = 1 - test_size
    train_count = int(len(X) * train_size)
    
    return X[:train_count], X[train_count:], Y[:train_count], Y[train_count:]
    
class neighbors:
    class KNeighborsClassifier:        
        def __init__(self,n_neighbors : int,p=2):
            #initialize
            self.X = None
            self.Y = None
            self.n_neighbors=n_neighbors
            self.p=p
            
        def fit(self,X_train: np.ndarray,Y_train: np.ndarray):
            # store X,Y_Train on fit
            self.X_train = X_train.astype(int)
            self.Y_train = Y_train.astype(int)
            
        def __minkowski_distance(self,x1 : np.ndarray,x2 : np.ndarray):
            return np.sum(np.abs(x1 - x2) ** self.p) ** (1 / self.p)

        def __kNN_majority(self,neighbors : np.ndarray):
            neighbors = neighbors[:self.n_neighbors] 
            neighbors = neighbors.transpose()[1] # use only y
            values, counts = np.unique(neighbors,return_counts=True)
            mode_value = np.argwhere(counts == np.max(counts))
            return values[mode_value].flatten().tolist()[0] # [0] mean if mode > 1 select first
            
        def predict(self,X : np.ndarray):
            X = X.astype(int)
            # each neighbor = [distance,y] 
            predictions = np.array([])
            for data in X:
                neighbors = np.empty((len(self.X_train), 2))
                for i,train_data in enumerate(self.X_train):
                    distance = self.__minkowski_distance(data,train_data)
                    neighbors[i] = [distance,self.Y_train[i]]
                neighbors = neighbors[neighbors[:, 0].argsort()]
                predictions = np.append(predictions,self.__kNN_majority(neighbors))
            return predictions
            
        def score(self,X:np.ndarray,Y:np.ndarray):
            X = X.astype(int)
            Y = Y.astype(int)
            predictions = self.predict(X)
            # print(X.shape,Y.shape,predictions)
            loss = np.sum(predictions != Y) / X.size
            return 1 - loss

In [2]:
# For your assignment, you have implement function "train_test_split" and
# module "neighbors" on your own. This means you will have to make the code working
# without any modification in cells 2-6

df = pd.read_csv('breast-cancer-wisconsin.csv')
df.replace('?', -99999, inplace=True)
df.drop(['id'], axis=1, inplace=True)


x = np.array(df.drop(['class'], axis=1))
y = np.array(df['class'])

In [3]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [4]:
h = neighbors.KNeighborsClassifier(n_neighbors=7, p=2)
h.fit(x_train, y_train)

In [5]:
new_x = np.array([4,6,5,6,7,8,4,9,1])
result = h.predict(new_x.reshape(1, -1))
print(result)

[4.]


In [6]:
print(h.score(x_test, y_test))

0.9968253968253968


# Summarize

1. feature sapce (X) 
   - 9 dimension (feature xi) relate with tumors dataset
2. label space (Y) 
   - 2 class (benign and malignant)
3. devide dataset with train_test_split() function
   - test_size = 20 %
   - train_size = 80 %
   - method : shuffle (fixed random seed)
4. k-Nearest Neighbors Algorithm
   - Minkowski distance 
     - p = 2 : Euclidean distance 
   - Training Step (fit) : store data points and its label
   - Predict Step (x : each data)
     - find distance from x to each data points (Neigbors)
     - sort ascending in set of neighbors
     - select k-NN from neighbors
     - predict mode of k-NN set
5. evaluate with zero/one loss metric
   - the accuracy is 1 - loss(X_test,Y_test)  